In [ ]:
# Import your libraries here
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
import wqet_grader
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository

Working with APIs

In [ ]:
ticker = "MTNOY"
output_size = "full"
data_type = "json"

url = (
   "https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?"
    "function=TIME_SERIES_DAILY&"
    f"symbol={ticker}&"
    f"outputsize={output_size}&"
    f"datatype={data_type}&"
    f"apikey={settings.alpha_api_key}"
)

print("url type:", type(url))
url

Create an HTTP request for the URL you created in the previous task. The grader will evaluate your work by looking at the ticker symbol in the "Meta Data" key-value pair in your response.


In [ ]:
response = requests.get(url = url)

print("response type:", type(response))

Get status code of your response and assign it to the variable response_code

In [ ]:
response_code = response.status_code

print("code type:", type(response_code))
response_code

Test-Driven Development -ETL

Create a DataFrame df_mtnoy with all the stock data for MTN. Make sure that the DataFrame has the correct type of index and column names. The grader will evaluate your work by looking at the row in df_mtnoy for 6 December 2021.

In [ ]:
response_data = response.json()
stock_data = response_data["Time Series (Daily)"]
df_mtnoy = pd.DataFrame.from_dict(stock_data,orient= "index", dtype =float)

print("df_mtnoy type:", type(df_mtnoy))
df_mtnoy.head()

In [ ]:
from data import AlphaVantageAPI

# Create instance of `AlphaVantageAPI` class
av = AlphaVantageAPI()
# Define Suzlon ticker symbol

ticker = "MTNOY"

# Use your `av` object to get daily data
df_mtnoy = av.get_daily(ticker=ticker)

print("df_mtnoy type:", type(df_mtnoy))
df_mtnoy.head()

Connect to the database whose name is stored in the .env file for this project. Be sure to set the check_same_thread argument to False. Assign the connection to the variable connection. The grader will evaluate your work by looking at the database location assigned to connection.

In [ ]:
connection = sqlite3.connect(database=settings.db_name, check_same_thread=False)
connection

In [ ]:
# Insert `MTNOY` data into database
# Import class definition
from data import SQLRepository

# Create instance of class
repo = SQLRepository(connection=connection)

response = repo.insert_table(table_name = ticker, records=df_mtnoy, if_exists="replace")

In [ ]:
%load_ext sql
%sql sqlite:////home/jovyan/work/ds-curriculum/080-volatility-forecasting-in-india/stocks.sqlite

In [ ]:
sql="SELECT * FROM MTNOY"
df_mtnoy_read = pd.read_sql(
    sql=sql, con=connection, parse_dates=["date"], index_col="date"
)

print("df_mtnoy_read type:", type(df_mtnoy_read))
print("df_mtnoy_read shape:", df_mtnoy_read.shape)
df_mtnoy_read.head()

Create a Series y_mtnoy with the 2,500 most recent returns for MTN.

In [ ]:
def wrangle_data(ticker,n_observations):

    """Extract table data from database. Calculate returns.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the stock (also table name in database).

    n_observations : int
        Number of observations to return.

    Returns
    -------
    pd.Series
        Name will be `"return"`. There will be no `NaN` values.
    """
    # Get table from database
    df =repo.read_table(table_name=ticker,limit=n_observations+1)

    # Sort DataFrame ascending by date
    df.sort_index(ascending = True, inplace=True)

    # Create "return" column
    df["return"] = df["close"].pct_change() * 100

    # Return returns
    return df["return"].dropna()

In [ ]:
y_mtnoy = wrangle_data(ticker="MTNOY", n_observations=2500)

print("y_mtnoy type:", type(y_mtnoy))
print("y_mtnoy shape:", y_mtnoy.shape)
y_mtnoy.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot returns for `df_suzlon` and `df_ambuja`
y_suzlon.plot(ax=ax, label="SUZLON")
y_ambuja.plot(ax=ax, label="AMBUJACEM")

# Label axes
plt.xlabel("Date")
plt.ylabel("Return")

# Add legend
plt.legend();

Calculate daily volatility for y_mtnoy, and assign the result to mtnoy_daily_volatility.

In [ ]:
import numpy as np

In [ ]:
mtnoy_annual_volatility = mtnoy_daily_volatility * np.sqrt(252)

print("mtnoy_annual_volatility type:", type(mtnoy_annual_volatility))
print("MTN Annual Volatility:", mtnoy_annual_volatility)

Create a time series line plot for y_mtnoy. Be sure to label the x-axis "Date", the y-axis "Returns", and use the title "Time Series of MTNOY Returns".

In [ ]:
# Create `fig` and `ax`
fig, ax = plt.subplots(figsize=(15, 6))

# Plot `y_mtnoy` on `ax`
y_mtnoy.plot(ax=ax)

# Add axis labels
plt.xlabel("Date")
plt.ylabel("Return")

# Add title
plt.title("Time Series of MTNOY Returns");

# Don't delete the code below 👇
plt.savefig("images/8-5-11.png", dpi=150)


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Create `fig` and `ax`
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared returns
plot_acf(y_mtnoy**2, ax=ax)

# Add axis labels
plt.xlabel("Lag [days]")
plt.ylabel("Correlation Coefficient");


# Add title
plt.title("ACF of MTNOY Squared Returns")

# Don't delete the code below 👇
plt.savefig("images/8-5-12.png", dpi=150)


Create a PACF plot of the squared returns for MTN. Be sure to label the x-axis "Lag [days]", the y-axis "Correlation Coefficient", and use the title "PACF of MTNOY Squared Returns"

In [ ]:
# Create `fig` and `ax`
fig, ax = plt.subplots(figsize=(15, 6))

# Create PACF of squared returns
plot_pacf(y_mtnoy**2, ax=ax)

# Add axis labels
plt.xlabel("Lag [days]")
plt.ylabel("Correlation Coefficient")

# Add title
plt.title("PACF of MTNOY Squared Returns")

# Don't delete the code below 👇
plt.savefig("images/8-5-13.png", dpi=150)


In [ ]:
#Create a training set y_mtnoy_train that contains the first 80% of the observations in y_mtnoy
cutoff_test = int(len(y_mtnoy) * 0.8)
y_mtnoy_train = y_mtnoy.iloc[:cutoff_test]  #walk forward validation

print("y_mtnoy_train type:", type(y_mtnoy_train))
print("y_mtnoy_train shape:", y_mtnoy_train.shape)
y_mtnoy_train.head()

Build Model

 Build and fit a GARCH model using the data in y_mtnoy. Try different values for p and q, using the summary to assess its performance. The grader will evaluate whether your model is the correct data type.

In [ ]:
from arch import arch_model
# Build and train model
model = arch_model(
    y_mtnoy_train,
    p=1, #changing from 3 to 2 because the values are not statistically significant
    q=1, #no of beta terms according to the pacf model
    rescale=False).fit(disp=0) # we dont want to see the whole details, hence display

print("model type:", type(model))

# Show model summary
model.summary() #we want the pvalues to be less than .05 to be statistically significant

In [ ]:
#Plot the standardized residuals for your model. Be sure to label the x-axis "Date", the y-axis "Value", and use the title "MTNOY GARCH Model Standardized Residuals"
# Create `fig` and `ax`
fig, ax = plt.subplots(figsize=(15, 6))

# Plot standardized residuals
model.std_resid.plot(ax=ax, label="standard residuals")

# Add axis labels
plt.xlabel("Date")
plt.ylabel("Value")


# Add title
plt.title("MTNOY GARCH Model Standardized Residuals")

# Don't delete the code below 👇
plt.savefig("images/8-5-16.png", dpi=150)


Create an ACF plot of the squared, standardized residuals of your model. Be sure to label the x-axis "Lag [days]", the y-axis "Correlation Coefficient", and use the title "ACF of MTNOY GARCH Model Standardized Residuals"

In [ ]:
# Create `fig` and `ax`
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared, standardized residuals
plot_acf(model.std_resid**2, ax=ax)

# Add axis labels
plt.xlabel("Lag [days]")
plt.ylabel("Correlation Coefficient")

# Add title
plt.title("ACF of MTNOY GARCH Model Standardized Residuals")

# Don't delete the code below 👇
plt.savefig("images/8-5-17.png", dpi=150)


Model Deployment

Change the fit method of your GarchModel class so that, when a model is done training, two more attributes are added to the object: self.aic with the AIC for the model, and self.bic with the BIC for the model. When you're done, use the cell below to check your work.

: How can you access the AIC and BIC scores programmatically? Every ARCHModelResult has an .aic and a .bic attribute.

In [ ]:
# Import `build_model` function
from main import build_model

# Build model using new `MTNOY` data
model=build_model(ticker="MTNOY", use_new_data=True)

# Wrangle `MTNOY` returns
model.wrangle_data(n_observations=2500)

# Fit GARCH(1,1) model to data
model.fit(p=1, q=1)

# Does model have AIC and BIC attributes?
assert hasattr(model, "aic")
assert hasattr(model, "bic")

In [ ]:
# Import `FitIn` class and `fit_model` function
from main import FitIn, fit_model

# Instantiate `FitIn` object
request = FitIn(ticker="MTNOY", use_new_data=False, n_observations=2500, p=1, q=1)

# Build model and fit to data, following parameters in `request`
fit_out = fit_model(request=request)

# Inspect `fit_out`
fit_out

Create a post request to hit the "/fit" path running at "http://localhost:8008". You should train a GARCH(1,1) model on 2500 observations of the MTN data you already downloaded. Pass in your parameters as a dictionary using the json argument. The grader will evaluate the JSON of your response

In [ ]:
# URL of `/fit` path
url = "http://localhost:8008/fit"
# Data to send to path
json = {
    "ticker":"MTNOY",
    "use_new_data":False,
    "n_observations":2000,
    "p":1,
    "q":1
}
# Response of post request
response = requests.post(url=url, json=json)

print("response type:", type(response))
print("response status code:", response.status_code)

Create a post request to hit the "/predict" path running at "http://localhost:8008". You should get the 5-day volatility forecast for MTN. When you're satisfied, submit your work to the grader.

In [ ]:
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {
    "ticker":"MTNOY",
    "n_days":5
}
# Response of post request
response = requests.post(url=url, json=json)

print("response type:", type(response))
print("response status code:", response.status_code)